<a href="https://colab.research.google.com/github/AP-047/RClass-Classification-by-Rational-Approximation/blob/main/notebooks/milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**1. Importing the MNIST dataset and selecting a subset**

In [19]:
import numpy as np
from keras.datasets import mnist

# Load the MNIST dataset
(X_full, y_full), (_, _) = mnist.load_data()

# Initialize containers for subsets
subset_X = []
subset_y = []

# Draw 100 images per digit (0-9)
for digit in range(10):
    digit_indices = np.where(y_full == digit)[0]
    selected_indices = digit_indices[:100]  # Select the first 100 images for this digit
    subset_X.append(X_full[selected_indices])
    subset_y.append(y_full[selected_indices])

# Combine the subsets
subset_X = np.vstack(subset_X)  # Stack all selected images
subset_y = np.hstack(subset_y)  # Stack all selected labels

# Normalize pixel values to [0, 1]
subset_X = subset_X.astype("float32") / 255.0

#____Testing____
print(f"Subset X shape (images): {subset_X.shape}")  # Expected shape: (1000, 28, 28)
print(f"Subset y shape (labels): {subset_y.shape}")  # Expected shape: (1000,)

Subset X shape (images): (1000, 28, 28)
Subset y shape (labels): (1000,)


**2. Dimensionality reduction using PCA**

In [20]:
from sklearn.decomposition import PCA

# Flatten the images to shape (1000, 784)
subset_X_flattened = subset_X.reshape(subset_X.shape[0], -1)

# Apply PCA to reduce to 50 components
pca = PCA(n_components=50)
subset_X_reduced = pca.fit_transform(subset_X_flattened)


#____Testing____
# Print the new shape of the dataset
print(f"Reduced X shape (after PCA): {subset_X_reduced.shape}")
# Print the explained variance ratio to verify the retained variance
explained_variance = np.sum(pca.explained_variance_ratio_)
print(f"Total explained variance retained: {explained_variance * 100:.2f}%")

#____Testing____
# Set the desired label (digit) to inspect
desired_label = 3
# Find the index of the first occurrence of the desired label
index = np.where(subset_y == desired_label)[0][0]
# Get the corresponding reduced vector from PCA
selected_vector = subset_X_reduced[index]
# Print the vector and its length
print(f"Vector for digit {desired_label}: {selected_vector}")
print(f"Length of the vector: {len(selected_vector)}")

Reduced X shape (after PCA): (1000, 50)
Total explained variance retained: 84.13%
Vector for digit 3: [ 2.1737843   2.2719154   0.5603799   3.6210668  -0.10026538  3.544359
  1.4235294   0.6697829  -0.00906836  0.59699255  0.10746204  0.10104614
 -0.7792362  -0.06476107  0.43508944 -0.04706734 -1.3331981   1.0558177
 -0.09923563 -0.35563737 -0.14666994  0.4774493   0.20649122 -1.3405517
 -0.19683918  0.67087394  0.12541449 -0.30570272 -0.6784954  -0.13758592
  0.20175792  0.1983411  -0.03441443 -0.1392019   0.46415824  0.6184237
  0.0841008   0.28841558  0.00554711  0.43878043  0.62503815  0.06517712
  0.21008125 -0.04887159  0.11035094 -0.03693259  0.13809957  0.04459683
 -0.04487007 -0.31000057]
Length of the vector: 50


**3. Implement simple multi-variate rational function**

In [23]:
import numpy as np

# Randomly initialize coefficients
np.random.seed(47)  # for reproducibility
nm_cf = np.random.rand(51)  # 50 + 1 bias term
dn_cf = np.random.rand(51)

# Define the rational function
def rational_function(x, nm_cf, dn_cf):
    # Add bias term (c_0 and d_0)
    P = np.dot(nm_cf[:-1], x) + nm_cf[-1]  # Linear numerator
    Q = np.dot(dn_cf[:-1], x) + dn_cf[-1]  # Linear denominator

    # Avoid division by zero
    if Q == 0:
        return float('inf')
    return P / Q

#____Testing____
sample_vector = subset_X_reduced[0] # First PCA-reduced vector
output = rational_function(sample_vector, nm_cf, dn_cf)
print(f"Output of the rational function: {output}")

Output of the rational function: -0.08683916685676232
